# Dist: PySnpTools for Distributions

This notebook illustrates the use of the PySnpTools 'DistReader' features.

See [PySnpTools's README.md](https://github.com/fastlmm/PySnpTools/blob/master/README.md) for for slides, video, installation instructions, documentation, and code.

> Version 0.4.22, September 13, 2020

### Contacts

* Email the developers at fastlmm-dev@python.org.
* [Join](mailto:fastlmm-user-join@python.org?subject=Subscribe) the user discussion and announcement list (or use [web sign up](https://mail.python.org/mailman3/lists/fastlmm-user.python.org)).
* [Open an issue](https://github.com/fastlmm/PySnpTools/issues) on GitHub.

# Sample

In [2]:
# Download a sample file
from pysnptools.util import example_file
bgen_file = example_file("pysnptools/examples/example.bgen")

# Read from the file
from pysnptools.distreader import Bgen
bgen = Bgen(bgen_file)          # Create a reader
probs0 = bgen[:,0].read().val   # Read 1st SNP
print(probs0.shape)             # Shape of the NumPy array

probs_all = bgen.read().val     # Read all variants
print(probs_all.shape)          # Shape of the NumPy array

(500, 1, 3)
(500, 199, 3)


# Set Up

pip install pysnptools

# Usage

In [3]:
from pysnptools.distreader import Bgen

We also need an example file, which you can download like this:

In [4]:
from pysnptools.util import example_file # Download and return local file name
bgen_file = example_file("pysnptools/examples/example.bgen")
bgen_file

'M:\\Temp\\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen'

# Documentation

Full documentation, with examples for every function discussed, can be found here:
https://fastlmm.github.io/PySnpTools/#module-pysnptools.distreader

# Reading a Bgen file

The idea behind PySnpTools is that we create 'readers' for genomic data. Readers can quickly tell about the individuals (also known as samples) and SNPs (also known as variants) in the file. The actual genomic data is not read until a .read() command is given. For efficiency, the readers try to read only the section of genomic data requested.

For example:

In [5]:
from pysnptools.distreader import Bgen
bgen = Bgen(bgen_file) #Create a Bgen reader, named 'bgen'
bgen

Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen')

In [6]:
print(bgen.shape)   #print the # of individuals and # of SNPs
print(bgen.iid[:5]) #print 1st 5 individuals
print(bgen.sid[:5]) #print 1st 5 SNPs
print(bgen.pos[:5]) #print chrom and position info for the 1st 5 SNPs

(500, 199)
[['0' 'sample_001']
 ['0' 'sample_002']
 ['0' 'sample_003']
 ['0' 'sample_004']
 ['0' 'sample_005']]
['SNPID_2,RSID_2' 'SNPID_3,RSID_3' 'SNPID_4,RSID_4' 'SNPID_5,RSID_5'
 'SNPID_6,RSID_6']
[[1.e+00 0.e+00 2.e+03]
 [1.e+00 0.e+00 3.e+03]
 [1.e+00 0.e+00 4.e+03]
 [1.e+00 0.e+00 5.e+03]
 [1.e+00 0.e+00 6.e+03]]


Reading from a Bgen reader produces a 'DistData' object. It contains a .val property that is a numpy array of distribution values.

In [7]:
distdata = bgen.read() #Read all the data from the file into the in-memory distdata object.
print(distdata) #The in-memory DistData
print(distdata.val[:2,:2]) #Show the distribution data for the first 2 individuals and first 2 SNPs

DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen'))
[[[       nan        nan        nan]
  [0.00506592 0.0020752  0.99285888]]

 [[0.02780236 0.00863674 0.9635609 ]
  [0.00964355 0.00268555 0.9876709 ]]]


DistData objects are also readers so they know the iid, sid, etc.distribution.

In [8]:
print(distdata.shape) #print the # of individuals and # of SNPs
print(distdata.iid[:5]) #print the first 5 individuals 

(500, 199)
[['0' 'sample_001']
 ['0' 'sample_002']
 ['0' 'sample_003']
 ['0' 'sample_004']
 ['0' 'sample_005']]


We can do all this in one line:

In [9]:
Bgen(bgen_file).read().val[:2,:2]

array([[[       nan,        nan,        nan],
        [0.00506592, 0.0020752 , 0.99285888]],

       [[0.02780236, 0.00863674, 0.9635609 ],
        [0.00964355, 0.00268555, 0.9876709 ]]])

By default, the sid name for a SNP will be something like

 > 'SNPID_2,RSID_2'.
 
If you only want the SNP id and not the RSID, for example,

> 'SNPID_2'

then create a Bgen reader with a `sid_function` of 'id'. (Likewise, if you want only the RSID, then 'rsid')

In [10]:
bgen2 = Bgen(bgen_file,sid_function='id') #Create a new reader that uses the SNP ids in the BGEN file. Ignore the RSID.
bgen2.sid[:5] #print the first 5 SNPs

array(['SNPID_2', 'SNPID_3', 'SNPID_4', 'SNPID_5', 'SNPID_6'], dtype='<U9')

By default, the .read() function will read values into a float64 NumPy array. That is much more precision that many Bgen files use. You'll save 50% of memory by instead reading with a dtype of 'float32'

In [11]:
distdata2 = bgen2.read(dtype='float32') #Read into a NumPy array with dtype 'float32'
distdata2.val.dtype #Show the dtype of the NumPy array

dtype('float32')

# Example Matrix Operation

In this example, we find the average distribution of each SNP and then list the 5 SNPs with the least and greatest entropy. To save memory, we read data from the BGEN file in batches of (no more than) 10 SNPs at a time.

In [12]:
import numpy as np
import pandas as pd
from pysnptools.distreader import Bgen

bgen3 = Bgen(bgen_file,sid_function='id') #Create a Bgen reader, named 'bgen3'. Use the SNP id, ignore the RSID
batch_size = 10
df_list = []
for start in range(0,bgen3.sid_count,batch_size): #Start with every 10th SNP
    distdata3 = bgen3[:,start:start+batch_size].read(dtype='float32')# Read batches of no more than 10 as a Numpy float32 array
    print(distdata3)  
    #Find the mean distribution for each SNP
    meandist = np.nanmean(distdata3.val,axis=0) #axis0 is iid, axis1 is SNP, axis3 is allele count
    meandist_as_string = [','.join(row) for row in meandist.astype('str')]
    shannon2 = -np.sum(meandist*np.log2(meandist),axis=1) #compute the entropy of each distribution
    #Create a Pandas dataframe of results
    df_list.append(pd.DataFrame({'sid':distdata3.sid,'shannon2':shannon2,'meandist':meandist_as_string}))
#Merge each batches Pandas dataframe, sort, and list the 5 most skewed SNPs    
df = pd.concat(df_list)
df.sort_values('shannon2',inplace=True)
df[:5]

DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen')[:,0:10])
DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen')[:,10:20])
DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen')[:,20:30])
DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen')[:,30:40])
DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen')[:,40:50])
DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen')[:,50:60])
DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen')[:,60:70])
DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/examples/example.bgen')[:,70:80])
DistData(Bgen('M:\Temp\carl/hashdown/de3b6b13abdfc441608d45fe030db4a0/pysnptools/

,sid,shannon2,meandist
7,SNPID_129,0.157309,"0.97979075,0.016114073,0.0040951488"
7,SNPID_29,0.157309,"0.0040951488,0.016114073,0.97979075"
1,SNPID_173,0.255571,"0.96289223,0.029730404,0.0073773814"
1,SNPID_73,0.255571,"0.0073773814,0.029730404,0.96289223"
1,SNPID_193,0.288233,"0.0066580246,0.03753765,0.95580435"


In [13]:
#Here are 5 most even distributions
df[-5:]

,sid,shannon2,meandist
7,SNPID_79,1.513786,"0.24601659,0.4847949,0.26918846"
5,SNPID_57,1.519135,"0.27145734,0.47892186,0.24962077"
5,SNPID_157,1.519136,"0.24962077,0.47892186,0.27145734"
5,SNPID_37,1.528185,"0.21882463,0.44508588,0.33608952"
5,SNPID_137,1.528185,"0.33608952,0.44508588,0.21882463"


# How fast does Bgen read?

The PySnpTool's Bgen reader is built on Danilo Horta's Bgen-reader-py package. On top of that, it includes some speed-ups that will go into the next version Danilo's tool.

Currently, on my machine, with 1000 individuals, Bgen can read from about 4000 SNPs per second, so about 4 million 3-value distributions per second. When the number of individuals goes to 250,000, Bgen continues to read about 4 million 3-value distributions per second.

# Can PySnpTools write Bgen files?

Yes, kind of. But you must install a 3rd party program and set an environment variable.

> To write BGEN files, install QCTool from https://www.well.ox.ac.uk/~gav/qctool/

> Set your QCTOOLPATH environment variable to the tool.

You'll likely need to restart this notebook so Python will see your new environment variable.

In [14]:
import os
if os.environ.get('QCTOOLPATH') is None:
    print("To write BGEN files, install QCTool from https://www.well.ox.ac.uk/~gav/qctool/")
    print("Set your QCTOOLPATH environment variable to the tool.")
os.environ['QCTOOLPATH']

'ubuntu run /mnt/m/qctool/build/release/qctool_v2.0.7'

 With QCTools installed, you can write BGEN files.
 
 Suppose we wish to write the first 10 SNPs to the example.bgen file to file example10.bgen.
 
 This example prints the size of the original example.bgen file and then it creates example10.bgen and then prints its size. 

In [15]:
from pysnptools.distreader import Bgen
import os
print(os.path.getsize(bgen_file))
example = Bgen(bgen_file) # a Bgen reader for the whole file
# create a reader for the subset of the file with all the individuals and the first 10 SNPs
# write that to a file
example10 = Bgen.write('example10.bgen',example[:,:10],bits=23)
print(os.path.getsize('example10.bgen'))
print(example10)

665108
30262
Bgen('example10.bgen')


Behind the scenes the 'write' method is generating a text-based *.gen file and then using QCTool to convert *.gen to *.bgen. The 'bits' option is the number of bits used to represent each probability in the distribution. The number ranges from 1 to 32.

* 1 - can only presesent probabilities of 0.0 and 1.0
* 8 - how UK Biobank writes their data (according to https://bitbucket.org/gavinband/bgen/wiki/BGEN_in_the_UK_Biobank)
* 16 - the default
* 23 - full resolution for Numpy float32
* 32 - the maximum
* (52) - The not-allowed value that would be needed for full resolution for Numpy float64.

# DistData, an In-Memory DistReader

When we read from a Bgen, the result is a DistData object that contains an iid, sid, pos, and NumPy val (with dimensions iid_count * sid_count * 3), where iid is the list of individuals, sid is the list of SNPs, and pos is an array of position information for each SNP.

We can also create a DistData object from scratch. For example,

In [16]:
import numpy as np
from pysnptools.distreader import DistData

iid = [('0','iid0'),('0','iid1')]
sid = ['snp0','snp1','snp2']
pos = [[1,0,1],[1,0,2],[1,0,3]] #chromosome, genetic distance, basepair distance
val = np.array([[[0,0,1],[.5,.25,.25],[.95,.05,0]],
                [[1,0,0],[44,44,44],[np.nan,np.nan,np.nan]]
               ],dtype='float32')
distdata = DistData(iid=iid,sid=sid,pos=pos,val=val,name='in-memory sample')
distdata

DistData(in-memory sample)

Two things to note:
* Missing data is represented as [np.nan,np.nan,np.nan]
* The values do not need to be proper, normalized probability distributions, so [44,44,44] is fine.

We can extract the usual info.

In [17]:
print(distdata.shape)
print(distdata.iid)
print(distdata.sid)
print(distdata.pos)
print(distdata.val)

(2, 3)
[['0' 'iid0']
 ['0' 'iid1']]
['snp0' 'snp1' 'snp2']
[[1 0 1]
 [1 0 2]
 [1 0 3]]
[[[ 0.    0.    1.  ]
  [ 0.5   0.25  0.25]
  [ 0.95  0.05  0.  ]]

 [[ 1.    0.    0.  ]
  [44.   44.   44.  ]
  [  nan   nan   nan]]]


PySnpTools doesn't currently offer a way to normalize the distributions (make them add up to 1), but this bit of code does the trick, changing [44,44,44] to [.33,.33,.33].

In [18]:
distdata.val /= distdata.val.sum(axis=2,keepdims=True)
distdata.val

array([[[0.        , 0.        , 1.        ],
        [0.5       , 0.25      , 0.25      ],
        [0.95      , 0.05      , 0.        ]],

       [[1.        , 0.        , 0.        ],
        [0.33333334, 0.33333334, 0.33333334],
        [       nan,        nan,        nan]]], dtype=float32)

We can write a DistData to Bgen format. When we read it back, the numbers change slightly because Bgen rounds number different than NumPy.

In [42]:
from pysnptools.distreader import Bgen

print(os.getcwd())#!!!CMK

bgen = Bgen.write('2x3samplecmk.bgen',distdata,bits=23) #write it
print(bgen)
bgen.read(dtype='float32').val #Read the data from disk
#del bgen

D:\OneDrive\programs\pstsgkit\doc\ipynb


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: '2x3samplecmk.bgen'

# Fancy Reads

PySnpTools provides nice ways to read just the data you want. For example, first or last individuals or SNPs. Every 20th SNP, just named SNPs, etc. Here are some examples.

In [20]:
from pysnptools.distreader import Bgen
example = Bgen(bgen_file,sid_function='id') #Create a reader

Read the 1st 10 individuals, first 5 SNPs, both, and then the last

In [21]:
distdata10i = example[:10,:].read(dtype='float32') # read just first 10 individuals across all SNPs
print(distdata10i.iid) #Print the iids
distdata10i.val.shape #Print the shape of the numpy array

[['0' 'sample_001']
 ['0' 'sample_002']
 ['0' 'sample_003']
 ['0' 'sample_004']
 ['0' 'sample_005']
 ['0' 'sample_006']
 ['0' 'sample_007']
 ['0' 'sample_008']
 ['0' 'sample_009']
 ['0' 'sample_010']]


(10, 199, 3)

In [22]:
distdata5s = example[:,:5].read(dtype='float32') # read all individuals from just first 5 SNPs
print(distdata5s.sid) #Print the sids
distdata5s.val.shape #Print the shape of the numpy array

['SNPID_2' 'SNPID_3' 'SNPID_4' 'SNPID_5' 'SNPID_6']


(500, 5, 3)

In [23]:
distdata10i5s = example[:10,:5].read(dtype='float32') # read 1st 10 individuals from just first 5 SNPs
print(distdata10i5s.iid) #Print the iids
print(distdata10i5s.sid) #Print the sids
distdata10i5s.val.shape #print the shape of the numpy array

[['0' 'sample_001']
 ['0' 'sample_002']
 ['0' 'sample_003']
 ['0' 'sample_004']
 ['0' 'sample_005']
 ['0' 'sample_006']
 ['0' 'sample_007']
 ['0' 'sample_008']
 ['0' 'sample_009']
 ['0' 'sample_010']]
['SNPID_2' 'SNPID_3' 'SNPID_4' 'SNPID_5' 'SNPID_6']


(10, 5, 3)

In [24]:
distdata_last10i5s = example[-10:,-5:].read(dtype='float32') # read last 10 individuals from just last 5 SNPs
print(distdata_last10i5s.iid) #Print the iids
print(distdata_last10i5s.sid) #Print the sids
distdata_last10i5s.val.shape #print the shape of the numpy array

[['0' 'sample_491']
 ['0' 'sample_492']
 ['0' 'sample_493']
 ['0' 'sample_494']
 ['0' 'sample_495']
 ['0' 'sample_496']
 ['0' 'sample_497']
 ['0' 'sample_498']
 ['0' 'sample_499']
 ['0' 'sample_500']]
['SNPID_196' 'SNPID_197' 'SNPID_198' 'SNPID_199' 'SNPID_200']


(10, 5, 3)

Here is how we read every 25th SNP.

In [25]:
distdata_every25s = example[:,::25].read(dtype='float32') # read all individuals every 25th SNP
print(distdata_every25s.sid) #Print the sids
distdata_every25s.val.shape #Print the shape of the numpy array

['SNPID_2' 'SNPID_27' 'SNPID_52' 'SNPID_77' 'SNPID_102' 'SNPID_127'
 'SNPID_152' 'SNPID_177']


(500, 8, 3)

We can read named SNPs

In [26]:
sid_list = ['SNPID_22','SNPID_122','SNPID_2']
distdata_2s = example[:,example.sid_to_index(sid_list)].read(dtype='float32') # read the listed SNPs
print(distdata_2s.sid) #Print the sids
distdata_2s.val.shape #print the shape of the numpy array

['SNPID_22' 'SNPID_122' 'SNPID_2']


(500, 3, 3)

In addition, DistReader read according to a list of index numbers or a list of Booleans. It is fine to "stack" such subsetting. In every case, the final read will (to the degree possible) only read the final data requested.

For example, here in one line, we selected a file, select every 2nd individual and every 25th SNP, and then select some
individuals by index number and SNPs by Boolean array, and then read and show the NumPy array.

In [27]:
from pysnptools.distreader import Bgen
Bgen(bgen_file,sid_function='id')[::2,::25]\
   [[1,2,4,8],[False,False,True,True,False,False,False,True,False]]\
   .read(dtype='float32').val

array([[[2.2027607e-01, 7.4981719e-02, 7.0474219e-01],
        [3.7230279e-03, 6.0117790e-03, 9.9026519e-01],
        [9.9026519e-01, 6.0117790e-03, 3.7230279e-03]],

       [[3.6285423e-02, 7.8558350e-01, 1.7813110e-01],
        [1.7882757e-02, 1.8309942e-04, 9.8193413e-01],
        [9.8193413e-01, 1.8309942e-04, 1.7882757e-02]],

       [[1.1596701e-03, 6.5612802e-03, 9.9227905e-01],
        [2.2156473e-02, 9.5220792e-01, 2.5635580e-02],
        [2.5635580e-02, 9.5220792e-01, 2.2156473e-02]],

       [[1.0857865e-01, 8.6313242e-01, 2.8288929e-02],
        [1.8127393e-02, 1.0497996e-02, 9.7137463e-01],
        [9.7137463e-01, 1.0497996e-02, 1.8127393e-02]]], dtype=float32)

# Distributions to Expected Values and Bed Data to Distributions

We've seen that a DistReader produces a 3-value probability distribution for every individual and SNP.
We can turn those 3-value distributions into a single expected value with the 'as_snp' method.

Here are the values from a Bgen reader, followed by the values from the a Bgen.as_snp() reader.

In [28]:
from pysnptools.distreader import Bgen
distreader = Bgen(bgen_file,sid_function='id')
distreader[:2,:2].read(dtype='float32').val

array([[[       nan,        nan,        nan],
        [0.00506592, 0.0020752 , 0.9928589 ]],

       [[0.02780236, 0.00863674, 0.9635609 ],
        [0.00964355, 0.00268555, 0.9876709 ]]], dtype=float32)

In [29]:
snpreader = distreader.as_snp()
snpreader[:2,:3].read(dtype='float32').val

array([[       nan, 1.987793  , 0.01550294],
       [1.9357585 , 1.9780273 , 0.99383545]], dtype=float32)

By default, the expected values will range from 0 to 2, but you can change that to, for example, a range of 0 to 1

In [30]:
snpreader1 = distreader.as_snp(max_weight=1)
snpreader1[:2,:3].read(dtype='float32').val

array([[       nan, 0.9938965 , 0.00775147],
       [0.96787924, 0.9890137 , 0.49691772]], dtype=float32)

We can also go in the opposite direction, in a way. First, let's download some Bed data.

In [31]:
import numpy as np
from pysnptools.snpreader import Bed

bed_file = example_file("pysnptools/examples/toydata.5chrom.*","*.bed")
bed = Bed(bed_file,count_A1=True)
print(bed.shape)   #print the # of individuals and # of SNPs
print(bed.iid[:5]) #print 1st 5 individuals
print(bed.pos[:5]) #print 1st chrom and position info for the 1st 5 SNPs

(500, 10000)
[['per0' 'per0']
 ['per1' 'per1']
 ['per2' 'per2']
 ['per3' 'per3']
 ['per4' 'per4']]
[[ 1. nan  1.]
 [ 1. nan  2.]
 [ 1. nan  3.]
 [ 1. nan  4.]
 [ 1. nan  5.]]


Bed values are 0,1,2 (or missing). They give count for allele A1. This says to read data for the first 3 individuals and the first 2 SNPs.  For the first individual and first SNP the value is "1" meaning allele A1 was seen once.

In [32]:
bed[:3,:2].read(dtype='float32').val

array([[1., 0.],
       [1., 0.],
       [0., 0.]], dtype=float32)

We can use 'as_dist' to create a distribution reader. Our A1 count of 1 because a distribution of [0,1,0], meaning 0 probability of an A1 count 0, 100% probability of an A1 count of 1, and 0 probability of a count of 2.

In [33]:
bedasdist = bed.as_dist()
bedasdist[:3,:2].read(dtype='float32').val

array([[[0., 1., 0.],
        [1., 0., 0.]],

       [[0., 1., 0.],
        [1., 0., 0.]],

       [[1., 0., 0.],
        [1., 0., 0.]]], dtype=float32)

# Other DistReader File Types

In addition to Bgen, we can also save distribution data into

* *.dist.memmap -- puts NumPy array both on disk and in memory for instant access (but bigger size than Bgen)
* *.dist.nzp -- save to compressed NumPy arrays on disk (medium size and medium access time)
* *.dist.hdf5 -- save arrays to Hdf5 format - like *.dist.npz but maybe more readable from other programs

These are fully documented with examples in https://fastlmm.github.io/PySnpTools/#module-pysnptools.distreader.

Let's re-do the matrix operation we did earlier, but using DistMemMap.

In [34]:
import numpy as np
import pandas as pd
import os
from pysnptools.distreader import Bgen, DistMemMap

bgen3 = Bgen(bgen_file,sid_function='id') #Create a Bgen reader, named 'bgen3'. Use the SNP id, ignore the RSID
print(os.path.getsize(bgen_file)) #print size of *.bgen file

#Convert to an on-disk/in-memory dist.memmap file. Under the covers, it automatically reads in batchs from Bgen.
distmm = DistMemMap.write('example2.dist.memmap',bgen3,dtype='float32') 
print(os.path.getsize('example2.dist.memmap')) #print size of *.dist.memmap

#Find the mean distribution for each SNP. This takes very little memory.
meandist = np.nanmean(distmm.val,axis=0) # Like a DistData, DistMemMap's have a 'val', the NumPy array
meandist_as_string = [','.join(row) for row in meandist.astype('str')]
shannon2 = -np.sum(meandist*np.log2(meandist),axis=1) #compute the entropy of each SNP's distribution
#Create a Pandas dataframe of results
df = pd.DataFrame({'sid':distmm.sid,'shannon2':shannon2,'meandist':meandist_as_string})
df.sort_values('shannon2',inplace=True)
df[:5]

665108
1212105


,sid,shannon2,meandist
127,SNPID_129,0.157309,"0.97979075,0.016114073,0.0040951488"
27,SNPID_29,0.157309,"0.0040951488,0.016114073,0.97979075"
171,SNPID_173,0.255571,"0.96289223,0.029730404,0.0073773814"
71,SNPID_73,0.255571,"0.0073773814,0.029730404,0.96289223"
191,SNPID_193,0.288233,"0.0066580246,0.03753765,0.95580435"


In this example, using DistMemMap gives us fast access, very low memory, and simplier code. The cost is the use of more diskspace. Also, the format is not known to other tools.

# More Features: On-the-fly random data and reading multiple files as one.

In this example, we'll generate random distributions, write to 22 bgen files and then read from all the files as one.

In [35]:
from pysnptools.distreader import DistGen #"Gen" here stands for 'generate'
from pysnptools.distreader import Bgen #"Gen" here stands for 'genetic'

distgen = DistGen(seed=123,iid_count=10,sid_count=10*1000) #Create a random generator for 10 individuals and 10K SNPs

#Ask about the generator and 'read' (generate) 4 distribution values
print(distgen.shape)   #print the # of individuals and # of SNPs
print(distgen.iid[:5]) #print 1st 5 individuals
print(distgen.pos[:5]) #print 1st chrom and position info for the 1st 5 SNPs
distgen[:2,:2].read(dtype='float32').val

(10, 10000)
[['0' 'iid_0']
 ['0' 'iid_1']
 ['0' 'iid_2']
 ['0' 'iid_3']
 ['0' 'iid_4']]
[[1.000000e+00 0.000000e+00 2.013180e+05]
 [1.000000e+00 0.000000e+00 5.076180e+05]
 [1.000000e+00 0.000000e+00 8.139180e+05]
 [1.000000e+00 0.000000e+00 1.120218e+06]
 [1.000000e+00 0.000000e+00 1.426518e+06]]


array([[[0.5758514 , 0.23658438, 0.18756425],
        [       nan,        nan,        nan]],

       [[0.4861365 , 0.26529667, 0.24856684],
        [0.42888874, 0.14374886, 0.42736238]]], dtype=float32)

DistGen doesn't really read from disk, but it acts like a reader. We can answer question such as: How many SNPs in chromosomes 22?

In [36]:
distgen[:,distgen.pos[:,0]==22].sid_count

183

We'll generate Bgen files from chrom 1 to 22.

In [37]:
%%time
for chrom in range(1,23): #generate Bgen files for chrom 1 to 22
    filename = 'chrom{0}.bgen'.format(chrom)
    Bgen.write(filename,distgen[:,distgen.pos[:,0]==chrom],bits=23)
    print('"{0}" has size {1}'.format(filename,os.path.getsize(filename)))

"chrom1.bgen" has size 97225
"chrom2.bgen" has size 95392
"chrom3.bgen" has size 79987
"chrom4.bgen" has size 75832
"chrom5.bgen" has size 72303
"chrom6.bgen" has size 68547
"chrom7.bgen" has size 63995
"chrom8.bgen" has size 58070
"chrom9.bgen" has size 54547
"chrom10.bgen" has size 54415
"chrom11.bgen" has size 54404
"chrom12.bgen" has size 54253
"chrom13.bgen" has size 42989
"chrom14.bgen" has size 41216
"chrom15.bgen" has size 40024
"chrom16.bgen" has size 37157
"chrom17.bgen" has size 34867
"chrom18.bgen" has size 32185
"chrom19.bgen" has size 25311
"chrom20.bgen" has size 27290
"chrom21.bgen" has size 19072
"chrom22.bgen" has size 21377
Wall time: 12.8 s


Now let's create a reader that reads from the 22 Bgen readers as if they were one reader

In [38]:
from pysnptools.distreader import _DistMergeSIDs #Experimental - may change in future
from pysnptools.distreader import Bgen

reader_list = [Bgen('chrom{0}.bgen'.format(chrom),sid_function='id') for chrom in range(1,23)]
mergereader = _DistMergeSIDs(reader_list,cache_file='mergecache.npz')
mergereader.col_count

FileNotFoundError: [Errno 2] No such file or directory: 'chrom1.bgen.metadata2.mmm.temp'

We read the distribution data for the first individual and report it as a Pandas dataframe.

In [ ]:
%%time
import pandas as pd

firstiid = mergereader[0,:].read(dtype='float32') #Read the data for the first individual across all SNPs
dist_as_string = [','.join(row) for row in firstiid.val[0,:].astype('str')]
df = pd.DataFrame({'chrom':firstiid.pos[:,0],'dist':dist_as_string})
df[-5:] #Show the last 5 rows